In [1]:
import pandas as pd
import numpy as np
from pandas import ExcelWriter
import csv
import os
from GLTaxSettingsByState import GLTaxSettingsByState

In [2]:
def excelSave(df_dict):    
    def write_df(dataframe, df_key):
        if dataframe is None:
            print("error w/ df")
        writer = ExcelWriter(df_key + ".xlsx", engine="xlsxwriter")
        dataframe.to_excel(writer, sheet_name=df_key + "_")
        writer.save()
    if isinstance(df_dict, type(dict())):
        for df_key in df_dict.keys():
            write_df(df_dict.get(df_key, None), df_key)
    write_df(df_dict)

In [3]:
csv_path = os.path.join(os.curdir, "../data/test_NAPGLDATA/CORE0918-1018.csv")

In [4]:
with open(csv_path) as csv_file:
    for i in range(0, 4):
        next(csv_file)
    df = pd.read_csv(csv_file)
    

In [5]:
df.head(1)

,Record Type::Number,CM Trx Type,TRX Timestamp Date,GL Posting Date,Trx Number,PaidToRcvd From,Description,TRX Amount,Originating Debit Amount,Originating Credit Amount,Main Account Segment,Segment2,Segment3,Segment4,Segment5,Segment6,Account Description
0,1::423,Bank Payment,2018-09-10,2018-09-10,3196,Scott Jarus,Area 1D Referee Management Meeting for MY2018,$270.01,$270.01,$0.00,5433,0,1,D,0,0,Clinic Training Expenses Referees


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26758 entries, 0 to 26757
Data columns (total 17 columns):
Record Type::Number          26758 non-null object
CM Trx Type                  26758 non-null object
TRX Timestamp Date           26758 non-null object
GL Posting Date              26758 non-null object
Trx Number                   26758 non-null object
PaidToRcvd From              26642 non-null object
Description                  15123 non-null object
TRX Amount                   26758 non-null object
Originating Debit Amount     26758 non-null object
Originating Credit Amount    26758 non-null object
Main Account Segment         26758 non-null int64
Segment2                     26758 non-null int64
Segment3                     26758 non-null int64
Segment4                     26758 non-null object
Segment5                     26758 non-null int64
Segment6                     26758 non-null int64
Account Description          26758 non-null object
dtypes: int64(5), object(12)


In [7]:
conditions = [df["Main Account Segment"] > 5000,
             df["Main Account Segment"] <= 5000]

outputs = ["Expense Account",
          "Revenue Account"]

In [8]:
res = np.select(conditions, outputs, "Other")
df["A/R"] = pd.Series(res)

In [9]:
exp_df = df.loc[df["A/R"] == "Expense Account"]

In [10]:
state_settings = GLTaxSettingsByState["CA"]
state_settings

{'4010': True,
 '4012': True,
 '4024': True,
 '4027': True,
 '5101': False,
 '5102': False,
 '5103': False,
 '5104': False,
 '5105': True,
 '5106': True,
 '5107': True,
 '5108': True,
 '5135': True}

In [11]:
conditions = list()
outputs = list()
for glcode in state_settings.keys():
    if state_settings.get(glcode):
        conditions.append(exp_df["Main Account Segment"] == int(glcode))
        outputs.append("Taxable")
res = np.select(conditions, outputs, "Non-Taxable")

In [12]:
dict_count = {}
for item in res:
    dict_count[item] = dict_count.get(item, 0) + 1

In [13]:
dict_count

{'Non-Taxable': 16274, 'Taxable': 913}

In [14]:
exp_df["Taxables?"] = pd.Series(res)

C:\Users\nabeel\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [15]:
exp_df

,Record Type::Number,CM Trx Type,TRX Timestamp Date,GL Posting Date,Trx Number,PaidToRcvd From,Description,TRX Amount,Originating Debit Amount,Originating Credit Amount,Main Account Segment,Segment2,Segment3,Segment4,Segment5,Segment6,Account Description,A/R,Taxables?
0,1::423,Bank Payment,2018-09-10,2018-09-10,3196,Scott Jarus,Area 1D Referee Management Meeting for MY2018,$270.01,$270.01,$0.00,5433,0,1,D,0,0,Clinic Training Expenses Referees,Expense Account,Non-Taxable
1,1::424,Bank Payment,2018-09-10,2018-09-10,3197,Rick Roberts,Domain name registration,$53.79,$53.79,$0.00,7515,0,1,D,0,0,Phone/Internet/Website,Expense Account,Non-Taxable
2,1::426,Bank Payment,2018-10-01,2018-10-01,3198,Mike Riley,Expenses for Advanced Coach Training Class,$0.00,$1085.34,$0.00,5432,0,1,D,0,0,Clinic Training Expenses Coaches,Expense Account,Non-Taxable
3,1::426,Bank Payment,2018-10-01,2018-10-01,3198,Mike Riley,Expenses for Advanced Coach Training Class // ...,$0.00,$0.00,$1085.34,5432,0,1,D,0,0,Clinic Training Expenses Coaches,Expense Account,Non-Taxable
4,1::427,Bank Payment,2018-10-01,2018-10-01,3199,AYSO,Manuals and Certificates for Advanced Coach Class,$562.93,$562.93,$0.00,5432,0,1,D,0,0,Clinic Training Expenses Coaches,Expense Account,Non-Taxable
5,1::428,Bank Payment,2018-10-01,2018-10-01,3200,Mike Riley,Expenses for Advanced Coach Training Class,$0.00,$1085.34,$0.00,5432,0,1,D,0,0,Clinic Training Expenses Coaches,Expense Account,Non-Taxable
6,1::428,Bank Payment,2018-11-04,2018-10-01,3200,Mike Riley,Expenses for Advanced Coach Training Class // ...,$0.00,$0.00,$1085.34,5432,0,1,D,0,0,Clinic Training Expenses Coaches,Expense Account,Non-Taxable
7,1::430,Bank Payment,2018-11-04,2018-10-01,3198,Mike Riley,Expenses for Advanced Coach Training Class,$1085.34,$1085.34,$0.00,5432,0,1,D,0,0,Clinic Training Expenses Coaches,Expense Account,Non-Taxable
8,1::438,Bank Payment,2018-12-12,2018-10-01,39,SERVICE FEE,NaN,$3.00,$3.00,$0.00,8305,0,1,D,0,0,Bank Fees,Expense Account,Non-Taxable
58,11::25,Customer Credit Note,2018-10-08,2018-10-08,24,Redlands AYSO Region 50,Redlands AYSO Challenge TeamCal-South Registra...,$0.00,$150.00,$0.00,5715,0,10,D,0,0,Payments to Referees (Playing Circuit),Expense Account,Non-Taxable
